In [3]:
import sys
import os
# Add the parent directory to the Python path

# Change the current working directory to /app
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')
market_name = 'backup_binance' #different from crypto_binance
timeframe='1m'
pair='BTC'

DATABASE_PATH: database/db/ohlcv_data.db


In [4]:
from backtest_engine.backtester import Backtester
from strategy.public.EmaStrat import EMAStrategy
import pandas as pd

def dummy_progress(progress: int, status: str) -> None:
        print(f"Progress: {progress}% - {status}")

# Initialize strategy with parameters
ema_strategy = EMAStrategy(
    fast_ema_period=10,
    slow_ema_period=100
)

# Create backtester with strategy
backtester = Backtester(
    market_name='crypto_binance',
    symbol_list=['ETH/BTC', 'LTC/BTC'],
    timeframe='15m',
    strategy_object=ema_strategy,
    strategy_type='multi',
    start_date=pd.Timestamp('2025-01-01'),
    end_date=pd.Timestamp('2025-02-13'),
    init_cash=100000,
    fees=0.0001,
    slippage=0.0001,
    size=0.1,
    cash_sharing=True,
    allow_partial=True,
    progress_callback=dummy_progress,
    pair='BTC'
)

Progress: 0% - Fetching data...
Len items in 'database/finstore/market_name=crypto_binance/timeframe=15m': 481
Progress: 5% - Reading existing data...


Progress: 25% - Running strategy...
Progress: 50% - Simulating portfolio...
Progress: 75% - Saving results...
Progress: 100% - Backtest complete.


In [5]:
dir(backtester.portfolio)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_seq',
 '_cash_sharing',
 '_close',
 '_config',
 '_fillna_close',
 '_iloc',
 '_indexing_kwargs',
 '_init_cash',
 '_loc',
 '_log_records',
 '_metrics',
 '_order_records',
 '_subplots',
 '_trades_type',
 '_wrapper',
 'alpha',
 'annual_returns',
 'annualized_return',
 'annualized_volatility',
 'asset_flow',
 'asset_returns',
 'asset_value',
 'assets',
 'benchmark_rets',
 'benchmark_returns',
 'benchmark_value',
 'beta',
 'build_metrics_doc',
 'build_subplots_doc',
 'call_seq',
 'calmar_ratio',
 'capture',
 'cash',
 'cash_flow',
 'cash_sharing',
 'close',
 'cond_value

In [42]:
pf = backtester.portfolio
pf.stats().to_frame(name='Value')
pf.value().max()
pf.cumulative_returns().index
pf.returns()
pf.trades.records_readable
pf.sharpe_ratio()
pf.sortino_ratio()
pf.cumulative_returns()
pf.save('test.pkl')
pf = pf.load('test.pkl')
pf.cumulative_returns()


timestamp
2025-01-30 17:00:00    0.000000
2025-01-30 17:15:00   -0.000020
2025-01-30 17:30:00   -0.000182
2025-01-30 17:45:00   -0.000847
2025-01-30 18:00:00   -0.000847
                         ...   
2025-02-10 01:45:00   -0.015534
2025-02-10 02:00:00   -0.016323
2025-02-10 02:15:00   -0.016498
2025-02-10 02:30:00   -0.015009
2025-02-10 02:45:00   -0.015622
Name: group, Length: 1000, dtype: float64

In [6]:
backtester.portfolio.plot_cum_returns()

FigureWidget({
    'data': [{'line': {'color': '#7f7f7f'},
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e2ace3c3-da72-4f53-8ea7-1bceca1a70c1',
              'x': array(['2025-01-30 17:00:00', '2025-01-30 17:15:00', '2025-01-30 17:30:00',
                          ..., '2025-02-10 02:15:00', '2025-02-10 02:30:00',
                          '2025-02-10 02:45:00'], dtype=object),
              'y': array([1.        , 1.00080906, 0.99919562, ..., 0.88785127, 0.89617617,
                          0.89398795])},
             {'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'f90c7823-fb81-43dd-8d33-4736d8961774',
              'x': array(['2025-01-30 17:00:00', '2025-01-30 17:15:00', '2025-01-30 17:30:00',
                          ..., '2025-02-10 02:15:00', '20

In [12]:
backtester.portfolio.get_trades().records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,ETH/BTC,2.883443e+05,2025-01-31 00:45:00,0.031183,0.899147,2025-01-31 04:15:00,0.030947,0.892336,-69.902154,-0.007774,Long,Closed,0
1,1,ETH/BTC,2.872523e+05,2025-01-31 06:30:00,0.031233,0.897179,2025-02-01 09:30:00,0.031887,0.915956,185.960269,0.020727,Long,Closed,1
2,2,ETH/BTC,3.154808e+05,2025-02-03 23:00:00,0.028393,0.895740,2025-02-04 05:15:00,0.027317,0.861807,-341.079572,-0.038078,Long,Closed,2
3,3,ETH/BTC,3.549939e+05,2025-02-04 11:45:00,0.027863,0.989112,2025-02-04 21:00:00,0.027637,0.981105,-82.039102,-0.008294,Long,Closed,3
4,4,ETH/BTC,3.127081e+05,2025-02-05 05:45:00,0.028383,0.887554,2025-02-06 11:45:00,0.028367,0.887064,-6.676318,-0.000752,Long,Closed,4
5,5,ETH/BTC,3.153457e+05,2025-02-07 09:45:00,0.028163,0.888102,2025-02-07 14:15:00,0.028007,0.883195,-50.844454,-0.005725,Long,Closed,5
6,6,ETH/BTC,3.237312e+05,2025-02-08 22:00:00,0.027373,0.886141,2025-02-08 22:30:00,0.027257,0.882403,-39.147518,-0.004418,Long,Closed,6
7,7,ETH/BTC,3.234700e+05,2025-02-09 01:30:00,0.027383,0.885749,2025-02-09 02:15:00,0.027287,0.882661,-32.649122,-0.003686,Long,Closed,7
8,8,ETH/BTC,3.205411e+05,2025-02-09 03:30:00,0.027623,0.885423,2025-02-09 14:45:00,0.027337,0.876272,-93.274889,-0.010535,Long,Closed,8
9,9,ETH/BTC,3.224214e+05,2025-02-09 15:15:00,0.027433,0.884490,2025-02-09 19:45:00,0.027317,0.880767,-38.996868,-0.004409,Long,Closed,9


In [4]:
from finstore.finstore import Finstore
symbol_list = ['ETH/BTC']
market_name = 'crypto_binance'
timeframe='15m'
finstore = Finstore(market_name=market_name, timeframe=timeframe, pair='BTC')

Len items in 'database/finstore/market_name=crypto_binance/timeframe=15m': 480


In [5]:
ohlcv_data = finstore.read.symbol_list(symbol_list)

In [6]:
from backtest_engine.backtester import Backtester
from strategy.public.EmaStrat import EMAStrategy
import pandas as pd

def dummy_progress(progress: int, status: str) -> None:
        print(f"Progress: {progress}% - {status}")

# Initialize strategy with parameters
ema_strategy = EMAStrategy(
    fast_ema_period=10,
    slow_ema_period=100
)

In [7]:
entries, exits, close_data, open_data = ema_strategy.run(ohlcv_data)

In [8]:
import vectorbtpro as vbt
pf = vbt.Portfolio.from_signals(
            close=close_data,
            open=open_data,
            entries=entries,
            exits=exits,
            direction='longonly',
            init_cash=10000,
            cash_sharing=True,
            size=1,
            size_type="valuepercent",
            fees=0.001,
            slippage=0.01,
            allow_partial=True,
            sim_start=pd.Timestamp('2025-01-01'),
        )

In [ ]:
pip install -U vectorbt

In [22]:
import vectorbt as vbt

pf = vbt.Portfolio.from_signals(
            close=close_data,
            open=open_data,
            entries=entries,
            exits=exits,
            direction='longonly',
            init_cash=10000,
            cash_sharing=True,
            size=1,
            fees=0.001,
            slippage=0.01,
            allow_partial=True,
            freq='15m',
        )

In [20]:
pf.get_trades()

In [23]:
pf.stats()

Start                         2025-02-07 14:30:00
End                           2025-02-11 21:30:00
Period                            4 days 07:15:00
Start Value                               10000.0
End Value                             9999.996493
Total Return [%]                        -0.000035
Benchmark Return [%]                    -2.221426
Max Gross Exposure [%]                   0.000277
Total Fees Paid                          0.000273
Max Drawdown [%]                         0.000035
Max Drawdown Duration             3 days 00:30:00
Total Trades                                    5
Total Closed Trades                             5
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                  0.0
Best Trade [%]                          -2.392645
Worst Trade [%]                         -2.818695
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                    -2.535397


In [9]:
pf.stats()

Start Index                   2025-02-07 14:30:00
End Index                     2025-02-11 21:30:00
Total Duration                    4 days 07:15:00
Start Value                               10000.0
Min Value                             8796.065207
Max Value                                 10000.0
End Value                             8796.065207
Total Return [%]                       -12.039348
Benchmark Return [%]                    -2.221426
Position Coverage [%]                   42.857143
Max Gross Exposure [%]                      100.0
Max Drawdown [%]                        12.039348
Max Drawdown Duration             3 days 00:45:00
Total Orders                                   10
Total Fees Paid                         93.969393
Total Trades                                    5
Win Rate [%]                                  0.0
Best Trade [%]                          -2.392645
Worst Trade [%]                         -2.818695
Avg Winning Trade [%]                         NaN
